In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install essentia
!pip install python_speech_features

     |████████████████████████████████| 11.1MB 214kB/s 
  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=765ef2e089701c21da19f23ca21f90121b33e4b86176abc64b73c494d136155c
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [ ]:
!unzip /content/drive/My\ Drive/data/o.zip
!unzip /content/drive/My\ Drive/data/other.zip
!unzip /content/drive/My\ Drive/data/w1.zip
!unzip /content/drive/My\ Drive/data/wheeze2.zip
!unzip /content/drive/My\ Drive/data/wheeze_and_crackle.zip

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import keras
from keras.layers import Dropout
from keras.layers import Activation
from shutil import copyfile
import gc
import os
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as signal
from scipy import stats
import python_speech_features
from essentia.standard import *
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
w = Windowing(type = 'hann')
spectrum = Spectrum()
mfcc = MFCC(highFrequencyBound = 8000, sampleRate = 16000)
gfcc = GFCC(highFrequencyBound = 8000, numberCoefficients = 40, sampleRate = 16000)
def get_feature(data):
  mfccs = []
  gfccs = []
  for frame in FrameGenerator(data, frameSize=1024, hopSize=512, startFromZero=True):
    mfcc_bands, mfcc_coeffs = mfcc(spectrum(w(frame)))
    gfcc_bands, gfcc_coeffs = gfcc(spectrum(w(frame)))
    mfccs.append(mfcc_coeffs)
    gfccs.append(gfcc_coeffs)
  test = np.asarray(gfccs)
  tam = np.asarray(mfccs)
  tam1 = python_speech_features.base.delta(tam, 10)
  tam2 = python_speech_features.base.delta(tam1, 10)
  return np.concatenate((test, tam, tam1, tam2), axis = 1)
def get_feature_all(dirs, X, Y, idx):
  for i in range(len(dirs)):
    dir = dirs[i] + "/"
    for filename in os.listdir(dir):
      try:
        rate, data = wav.read(dir + filename)
        test = [0]
        test[0] = idx[i]
        if idx[i] == 0:
          for t in range(0, 20):
            X.append(get_feature(np.roll(data, t * len(data)//20)))
            Y.append(test)
        elif idx[i] == 1:
          for t in range(0, 4):
            X.append(get_feature(np.roll(data, t * len(data)//4)))
            Y.append(test)
      except Exception as e:
        print(e)

In [0]:
dirs = ["/content/wheeze2/wheeze", "/content/other", "/content/w", "/content/wheeze_and_crackle", "/content/o"]
labels = [0, 1, 0, 0, 1]
X = []
y = []
get_feature_all(dirs,X, y, labels )

In [0]:
X = np.asarray(X)
y = np.asarray(y)
X1 = X
X = stats.zscore(X)

In [0]:
def create_model():
  model = Sequential()
  model.add(LSTM(64, return_sequences=True, input_shape=(93, 79)))
  model.add(LSTM(64))
  model.add(Dense(1, activation = "sigmoid"))
  model.compile(loss='binary_crossentropy', optimizer= "adam", metrics=['accuracy'])
  return model

In [0]:
def train(X_aug, y_aug, epochs = 40):
  i = 1
  for train_index, test_index in kf.split(X_aug):
    print("Cross Valid: " + str(i))
    model = create_model()
    model.fit(X_aug[train_index], y_aug[train_index], epochs = epochs, batch_size = 64)
    y_pred = model.predict(X_aug[test_index])
    y_pred = [1 * (x[0]>=0.5) for x in y_pred]
    print(confusion_matrix(y_aug[test_index], y_pred))
    gc.collect()
    i += 1

In [ ]:
#use same randomstate 28
kf = KFold(n_splits=5, shuffle= True, random_state=28)

In [21]:
train(X, y, 35)

Cross Valid: 1
Epoch 1/35
24739/24739 [==============================] - 80s 3ms/step - loss: 0.2799 - acc: 0.8893
Epoch 2/35
24739/24739 [==============================] - 79s 3ms/step - loss: 0.0852 - acc: 0.9706
Epoch 3/35
24739/24739 [==============================] - 79s 3ms/step - loss: 0.0371 - acc: 0.9887
Epoch 4/35
24739/24739 [==============================] - 78s 3ms/step - loss: 0.0245 - acc: 0.9927
Epoch 5/35
24739/24739 [==============================] - 78s 3ms/step - loss: 0.0223 - acc: 0.9930
Epoch 6/35
24739/24739 [==============================] - 79s 3ms/step - loss: 0.0137 - acc: 0.9957
Epoch 7/35
24739/24739 [==============================] - 78s 3ms/step - loss: 0.0146 - acc: 0.9956
Epoch 8/35
24739/24739 [==============================] - 78s 3ms/step - loss: 0.0127 - acc: 0.9964
Epoch 9/35
24739/24739 [==============================] - 78s 3ms/step - loss: 0.0053 - acc: 0.9987
Epoch 10/35
24739/24739 [==============================] - 79s 3ms/step - loss: 0.010

In [0]:
def train(X_aug, y_aug, epochs = 40):
  i = 1
  for train_index, test_index in kf.split(X_aug):
    if i < 4:
      i += 1
      continue
    print("Cross Valid: " + str(i))
    model = create_model()
    model.fit(X_aug[train_index], y_aug[train_index], epochs = epochs, batch_size = 64, verbose = 0)
    y_pred = model.predict(X_aug[test_index])
    y_pred = [1 * (x[0]>=0.5) for x in y_pred]
    print(confusion_matrix(y_aug[test_index], y_pred))
    gc.collect()
    i += 1

In [10]:
train(X, y, 30)

Cross Valid: 4





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








[[3138    2]
 [   3 3042]]
Cross Valid: 5
[[3046    4]
 [   4 3130]]
